# 1. Setup
field = FiniteField(...)
poly = Polynomial([1, 2, 3])  # 원본 다항식
zk_poly = ZKFRISender(poly).blind_polynomial()

# 2. FRI Commitment
fri_sender = FullFRISender(zk_poly)
merkle_roots = fri_sender.commit_phase()

# 3. Interactive Proof
verifier_challenges = [field.random_element() for _ in range(num_rounds)]
proofs = fri_sender.generate_proof(verifier_challenges)

# 4. Verification
assert FullFRIVerifier(merkle_roots).verify(proofs, verifier_challenges)

In [6]:
from functools import lru_cache

class FiniteField:
    def __init__(self, value, prime):
        if not self.is_prime(prime):
            raise ValueError(f"{prime} is not a prime")
        self.value = value % prime
        self.prime = prime

    @staticmethod
    def is_prime(n: int) -> bool:
        # 간소화된 소수 검증 (실제로는 Miller-Rabin 구현)
        if n < 2:
            return False
        for p in [2,3,5,7,11]:
            if n % p == 0:
                return n == p
        return True  # 임시 처리

    def __add__(self, other: 'FiniteField') -> 'FiniteField':
        if self.prime != other.prime:
            raise ValueError('Cannot add two numbers in different Fields')
        return FiniteField(self.value + other.value, self.prime)

    def __sub__(self, other: 'FiniteField') -> 'FiniteField':
        if self.prime != other.prime:
            raise ValueError('Cannot subtract two numbers in different Fields')
        return FiniteField(self.value - other.value, self.prime)

    def __mul__(self, other: 'FiniteField') -> 'FiniteField':
        if self.prime != other.prime:
            raise ValueError('Cannot multiply two numbers in different Fields')
        return FiniteField(self.value * other.value, self.prime)

    def __pow__(self, exponent: int) -> 'FiniteField':
        return FiniteField(pow(self.value, exponent, self.prime), self.prime)

    def __neg__(self):
        return FiniteField(-self.value, self.prime)

    def __eq__(self, other: object) -> bool:
        if not isinstance(other, FiniteField):
            return False
        return self.value == other.value and self.prime == other.prime

    def __ne__(self, other: object) -> bool:
        return not (self == other)

    def __rmul__(self, scalar: int) -> 'FiniteField':
        return FiniteField((scalar * self.value) % self.prime, self.prime)

    def __truediv__(self, other: 'FiniteField') -> 'FiniteField':
        if self.prime != other.prime:
            raise ValueError('Cannot divide two numbers in different Fields')
        return FiniteField(self.value * pow(other.value, -1, self.prime), self.prime) 

    def __repr__(self):
        return f"{self.value} (mod {self.prime})"

    @classmethod
    def random(cls, prime: int) -> 'FiniteField':
        return cls(random.randint(0, prime-1), prime)

    @property
    @lru_cache(maxsize=None)
    def inverse(self) -> 'FiniteField':
        if self.value == 0:
            raise ZeroDivisionError
        return FiniteField(pow(self.value, -1, self.prime), self.prime)

In [7]:
prime = 17
a = FiniteField(5, prime)
b = FiniteField(12, prime)

# 덧셈/뺄셈
assert (a + b).value == (5 + 12) % 17  # 17 → 0
assert (a - b).value == (5 - 12) % 17  # -7 → 10

# 곱셈/나눗셈
c = a * b
assert c.value == (5 * 12) % 17  # 60 → 9
d = c / a
assert d == b  # 9 / 5 = 12 mod 17

# 지수 연산
assert (a**3).value == (5**3) % 17  # 125 → 6

In [11]:
import sympy as sp
import galois

class UnivariatePolynomial:
    def __init__(self, coefficients: list):
        self.coefficients = coefficients.copy()
        self._strip_zeros()

    def _strip_zeros(self):
        """Trailing zeros 제거 (예: [1, 2, 0, 0] → [1, 2])"""
        while len(self.coefficients) > 0 and self.coefficients[-1] == 0:
            self.coefficients.pop()

    def to_sympy(self, symbol='x'):
        x = sp.symbols(symbol)
        return sum(coeff * x**i for i, coeff in enumerate(self.coefficients))

    def to_galois(self, field):
        """높은 차수부터 저장하는 galois.Poly 호환 변환"""
        return galois.Poly(self.coefficients[::-1], field=field)  # 역순

    def evaluate(self, x):
        return sum(coeff * (x**i) for i, coeff in enumerate(self.coefficients))

    def __add__(self, other):
        max_degree = max(self.degree(), other.degree())
        padded_self = self.coefficients + [0]*(max_degree - self.degree())
        padded_other = other.coefficients + [0]*(max_degree - other.degree())
        return UnivariatePolynomial([a+b for a,b in zip(padded_self, padded_other)])

    def degree(self):
        return len(self.coefficients) - 1 if self.coefficients else 0

    def __repr__(self):
        terms = []
        for i, coeff in enumerate(self.coefficients):
            if coeff != 0:
                term = f"{coeff}x^{i}" if i != 0 else f"{coeff}"
                terms.append(term)
        return " + ".join(terms) if terms else "0"